In [1]:
# ## inputs
water_xyz = water = """3

H\t0.7493682\t0.0\t0.4424329
O\t0.0\t0.0\t-0.1653507
H\t-0.7493682\t0.0\t0.4424329
#     """
geometry = water_xyz[3:]

# methane = [
# ['C', (0.0000, 0.0000, 0.0000)],
# ['H', (0.5288, 0.1610, 0.9359)],
# ['H', (0.2051, 0.8240, -0.6786)],
# ['H', (0.3345, -0.9314, -0.4496)],
# ['H', (-1.0685, -0.0537, 0.1921)]
# ]

# geometry = methane

In [13]:
from nbed.embed import nbed_driver


n_active_atoms=2
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
convergence = 1e-10
charge = 0
mu_level_shift=1e6
run_ccsd = True
run_fci = True
max_ram = 8_000


loc_method = "spade" #"spade" #"pipekmezey" "ibo" "boys"
run_mu_shift = True
run_huzinaga = True


exp_obj = nbed_driver(
    geometry ,
    n_active_atoms,
    basis,
    xc_functional,
    convergence,
    localization_method= loc_method,
    run_mu_shift = run_mu_shift,
    run_huzinaga = run_huzinaga,
    mu_level_shift = mu_level_shift,
    run_ccsd_emb = run_ccsd,
    max_ram_memory = max_ram,
    run_fci_emb=run_fci,
    perform_global_fci=True,
    get_full_system_H= True)


global FCI: -75.0153524759469


In [14]:
exp_obj.Generate_embedded_H()

embedded HF energy MU_SHIFT: -61.19993869044616, converged: True
CCSD Energy MU shift:
	%s -73.81211058487924
FCI Energy MU shift:
	%s -80.35675362397947
embedded HF energy HUZINAGA: -61.199938650398636, converged: True
CCSD Energy HUZ:
	%s -73.81211057558394
FCI Energy HUZ:
	%s -80.35675352175028


In [15]:
exp_obj.print_summary()

********************************************************************************
                         Summary of Embedded Calculation                        
********************************************************************************
global (cheap) DFT calculation -75.2778848340467
********************************************************************************
  Huzinaga calculation
Total energy - active system at RHF level: -73.77379005880088
Total energy - active system at CCSD level: -73.81211057558394
Total energy - active system at FCI level: -80.35675352175028
length of huzinaga embedded fermionic Hamiltonian: 5257
number of qubits required: 12
********************************************************************************
  Mu shift calculation
Total energy - active system at RHF level: -73.77379009888423
Total energy - active system at CCSD level: -73.81211058487924
Total energy - active system at FCI level: -80.35675362397947
length of mu embedded fermionic Hamilton

In [5]:
from nbed.embed import get_qubit_hamiltonian

qubit_H_mu = get_qubit_hamiltonian(exp_obj.molecular_ham_MU, 'jordan_wigner')
qubit_H_huz = get_qubit_hamiltonian(exp_obj.molecular_ham_HUZ, 'jordan_wigner')

qubit_H_full_system = get_qubit_hamiltonian(exp_obj.full_system_molecular_ham, 'jordan_wigner')

In [6]:
from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [7]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [8]:
import numpy as np
mu_full_quantum = eig_vals_emb_mu[0] 
huz_full_quantum = eig_vals_emb_huz[0]

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

-76.05654313527407
-74.91098548292882
1.1455576523452464


In [9]:
N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb_huz)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

6